# 情報科学演習II アルゴリズムとデータ構造II 第2週 2冊目

#### 第2週目次

1. ダイクストラのアルゴリズムの簡単な実装（1冊目）
2. 実装のクラス化（2冊目）
3. 優先度付きキュー（3冊目）
4. ダイクストラのアルゴリズムの効率的な実装（4冊目）

## 2. 実装のクラス化

1章で作成したdijkstra関数とそこから呼び出される関数をクラスにまとめます．

1章と同じノートブックだと同一の仮想マシンで混乱するため，別のノートブックです．
1冊目を横に並べて表示しておくことをお勧めします．

次の2つのセルは準備ですので，実行してください．
二つ目のセルで必要なモジュールを`import`し，いくつかの関数を定義しておきます．

In [ ]:
!apt install libgraphviz-dev
!pip install pygraphviz
# 最終行は Successfully installed pygraphviz-x.xx
# x.xxの部分は1.13などのバージョン番号

In [ ]:
# 準備
import networkx as nx
import math
import pygraphviz as pgv
from IPython.display import SVG, display_svg
from test import support # sortdict関数

log = False # log = Trueにすると途中経過が出力される

def print_graph(G): # GはnetworkxのGraphクラス (networkx.classes.graph.Graphクラス)
  print('print_graph:')
  print("  nodes", list(G.nodes))
  # print('  edges', list(G.edges))
  print('  edges', list(G.edges.data())) # 辺の重みも出力

def draw_graph(G, filename = 'tmp'):
  A = nx.nx_agraph.to_agraph(G)
  for edge in G.edges(data="weight"):
    u, v, d = edge
    A.get_edge(u, v).attr['label'] = d

  A.layout(args='-Nshape=circle')  # default prog='neato' 
  A.draw(F'{filename}.svg')
  display_svg(SVG(F'{filename}.svg'))

def sample_graph1():
  G = nx.Graph() # 無向グラフ
  for i,j,w in [[0, 1, 5], [0, 4, 9], [0, 7, 8], [1, 2, 12], [1, 3, 15], 
                [1, 7, 4], [2, 3, 3], [2, 6, 11], [3, 6, 9], [4, 5, 4], 
                [4, 7, 5], [5, 2, 1], [5, 6, 13], [7, 2, 7], [7, 5, 6]]:
    G.add_edge(i, j, weight=w) # weightはnetworkx, labelはgraphviz
  L = nx.get_edge_attributes(G, 'weight')
  print('edge length', L)
  return G

G1 = sample_graph1()
L1 = nx.get_edge_attributes(G1, 'weight')
answer_d1 = {0: 0, 1: 5, 2: 14, 3: 17, 4: 9, 5: 13, 6: 25, 7: 8}
    
def sample_graph2():
  G = nx.Graph() # 無向グラフ
  for i,j,w in [[7, 5, 6], [7, 2, 7], [5, 6, 13], [5, 2, 1], [4, 7, 5], 
                [4, 5, 4], [3, 6, 9], [2, 6, 11], [2, 3, 3], [1, 7, 4], 
                [1, 3, 15], [1, 2, 12], [0, 7, 8], [0, 4, 9], [0, 1, 5]]:
    G.add_edge(i, j, weight=w) # weightはnetworkx, labelはgraphviz
  L = nx.get_edge_attributes(G, 'weight')
  print('edge length', w)
  return G

G2 = sample_graph2()
L2 = nx.get_edge_attributes(G2, 'weight')
answer_d2 = {0: 0, 1: 5, 2: 14, 3: 17, 4: 9, 5: 13, 6: 25, 7: 8}

次のセルはサンプルのグラフG1とG2を図示するコードですので，実行しなさい．

In [ ]:
print('G1')
print_graph(G1)
draw_graph(G1)

print('G2')
print_graph(G2)
draw_graph(G2)

### 2.1 クラス化する

先ほどのノートブックで作成したダイクストラのアルゴリズムの関数群（dijkstra, initialize_single_source, insert, extract_min, relax）を次のコードセルの`FirstImplementation`クラスにまとめなさい．

このクラスの一部を作成してありますので，残りの部分を書く必要があります．

- クラス名は`FirstImplementation`．
- 1.2節で作った5つの関数をこのクラス内で定義する．
  
  字下げした範囲がクラス．
- 2つの辞書`d`と`pi`，リスト`Q`をこのクラスに入れる．
  
  初期化関数`__init__`内で辞書`d`と`pi`，リスト`Q`を宣言する．
  関数内でこれらの変数を読み書きするときは`self.d`のように変数名の前に`self.`をつけて使う．
- 関数を定義する際は，第1引数に`self`を<u>追加する．</u>
  
  `self`は自身のオブジェクトを指す．例：`insert`関数の定義の部分．

  `def insert(self, u):`
  
- 関数を呼び出す側では関数名の前に`self.`をつける．
  
  関数呼び出しのとき第1引数`self`は<u>書かない</u>．
  例：`dijkstra`関数内で`insert`関数を呼び出している部分．

  `self.insert(u) # Qに頂点uを加える`
  
- このクラスのオブジェクトを初期化する関数は`__init__`．
  参考：[チュートリアル __init__](https://docs.python.org/ja/3/reference/datamodel.html#object.__init__)．Javaのコンストラクタに対応するのが`__init__`関数です．
- 字下げ（インデント）に注意しましょう．
  クラス全体を字下げします．
  関数全体も字下げします．

作成済みの関数
- `__init__`
- `dijkstra`

書き加える必要がある関数
- `initialize_single_source(G)`
- `insert(u)`
- `extract_min()`
- `relax(u, v, L)`

1.2節の関数と同じ処理になるようにこれら4つの関数を書きましょう．
`self`がつくことと，字下げが増えるること以外は1.2節のままです．

**書き換えを楽にする技法**
`relax`関数内で辞書`d`を何度も使います．
関数の先頭で次のように`d = self.d`と書いておくと，その関数内では

`self.d[u]`や`self.d[v]`と何度も書く代わりに，

`d[u]`や`d[v]`のように短く書けるのでおすすめです．

**編集の技法**
複数行の字下げをまとめて増やしたい場合は，コードセルの行を選んだ状態でTabキーを押します．
逆に，複数行の字下げをまとめて減らししたい場合は，コードセルの行を選んだ状態でShift+Tabキーを押します．

**練習**
`dijkstra`関数から呼び出される4つの関数を書きなさい．

In [ ]:
class FirstImplementation:
    
  def __init__(self):
    self.d = {}   # d[u]は頂点uの距離
    self.Q = []   # 頂点が距離順に格納されたヒープ．Q[0]が最小距離の頂点
    self.pi = {} # pi[u]は頂点uへの最短経路上で，頂点uの直前にある頂点

  # sから各頂点への最短距離を返す
  # G: グラフ, L: Gの各辺の長さの辞書, s: 始点とする頂点
  def dijkstra(self, G, L, s):
    d = self.d # この関数内ではd[s]と書けばself.d[s]を読み書きできる
    Q = self.Q
    self.initialize_single_source(G) # すべての頂点uについて距離d[u]を初期化する
    S = set() # Sは空集合
    
    for u in list(G.nodes): # Vに含まれるそれぞれの頂点uに対して
      self.insert(u) # Qに頂点uを加える
    
    d[s] = 0

    # 最短距離を求める
    while len(Q) > 0: # Qが空集合になるまで
      u = self.extract_min()   #   Qから最小値の頂点uを取り出す
      S.add(u)       #   Sに頂点uを加える
      for v in list(G.adj[u]): #   uに隣接するそれぞれの頂点に対して
        if not v in S: # まだSに含まれていないならば
          self.relax(u, v, L) #     頂点vへの二つの経路のうち短いものを残しキューの並びも調整する
    return self.d

  # すべての頂点uについて距離d[u]を初期化する
  def initialize_single_source(self, G):
    ....

  # Qに頂点uを加える
  def insert(self, u):
    ....

  # Qから最小値の頂点uを取り出す．
  def extract_min(self):
    return ....

  # 頂点vへの二つの経路のうち短いものを残す
  def relax(self, u, v, L):
    ....

### 2.2 クラス化したものをテストする

次のコードセルでテストします．
うまくFirstShortestPathクラスが作れたか，以下のコードセルを実行して，
グラフG1，始点0について1章と同じ結果が得られることを確かめよ．

`d = {0: 0, 1: 5, 2: 14, 3: 17, 4: 9, 5: 13, 6: 25, 7: 8}`が正解です．

In [ ]:
c = FirstImplementation()
d = c.dijkstra(G1, L1, 0)
print("d =", support.sortdict(d)) # 距離の辞書dを出力（頂点でソートして出力)
if d == answer_d1:
  print('正解です')
else:
  print('間違っています')

上のコードの最初の2行について解説します．

`c = FirstImplementation()`は`FirstImplementation`クラスのオブジェクトを生成して，変数`c`に格納します．
生成のときに呼び出されているのは`__init__(self)`関数です．
生成されたオブジェクト`c`には`d`と`Q`と`pi`のデータが保持されます．
これらのデータは`dijkstra`関数や`insert`関数などで読み書きされます．

`d = c.dijkstra(G1, L1, 0)`はオブジェクト`c`の`dijkstra`関数を呼び出して，その戻り値を`d`に格納します．
`c`は`FirstImplementation`クラスのオブジェクトですから，クラス内に定義した`dijkstra`関数が呼び出されます．
呼び出し元の`G1, L1, 0`は単一始点最短距離問題のグラフ，辺の長さの辞書，始点として`dijkstra`関数に渡されます．
`dijkstra`関数は各頂点の始点0からの最短距離の入った辞書を返します．
この辞書が`d`に格納されます．

もう一つのグラフG2，始点0についても以下のコードセルを実行して確かめましょう．

`d = {0: 0, 1: 7, 2: 12, 3: 15, 4: 9, 5: 11, 6: 23, 7: 5}`が正解です．

In [ ]:
c = FirstImplementation()
d = c.dijkstra(G2, L2, 0)
print("d =", support.sortdict(d)) # 距離の辞書dを出力（頂点でソートして出力)
if d == answer_d2:
  print('正解です')
else:
  print('間違っています')
answer_d2

最後にグラフG1について，始点を変えながら動作を確かめましょう．

In [ ]:
for s in range(G1.order()):
  d = FirstImplementation().dijkstra(G1, L1, s)
  answer_d = nx.single_source_dijkstra_path_length(G1, s)
  if d == answer_d:
    print(F'始点が{s}のときすべての頂点の距離が正しく求められました．')
  else:
    print(F'始点が{s}のとき距離が間違っている頂点があります．')
    print()
    for u, du in d.items():
      if du != answer_d[u]:
        print(F'始点{s}から頂点{u}への最短距離は{answer_d[u]}ですが，{du}と求めています．')

draw_graph(G1)